In [1]:
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import numpy as np
import pandas as pd
import random

In [9]:
now_dt = datetime.now()
now_ts = datetime.timestamp(now_dt)

print(now_dt)
print(now_ts)
print(datetime.utcfromtimestamp(now_ts))

2020-03-30 18:14:13.202840
1585563253.20284
2020-03-30 10:14:13.202840


In [59]:
ts_list = [now_ts]
ts      = now_ts
for i in range(0, 10):
    ts = ts + random.randrange(20, 40)
    ts_list.append(ts)

ts_list

[1585563253.20284,
 1585563290.20284,
 1585563315.20284,
 1585563338.20284,
 1585563373.20284,
 1585563397.20284,
 1585563434.20284,
 1585563468.20284,
 1585563494.20284,
 1585563527.20284,
 1585563550.20284]

In [60]:
d  = {'TIMESTAMP': ts_list, 'DATETIME': [datetime.utcfromtimestamp(i) for i in ts_list]}
df = pd.DataFrame(data=d)
df

,TIMESTAMP,DATETIME
0,1.585563e+09,2020-03-30 10:14:13.202840
1,1.585563e+09,2020-03-30 10:14:50.202840
2,1.585563e+09,2020-03-30 10:15:15.202840
3,1.585563e+09,2020-03-30 10:15:38.202840
4,1.585563e+09,2020-03-30 10:16:13.202840
5,1.585563e+09,2020-03-30 10:16:37.202840
6,1.585563e+09,2020-03-30 10:17:14.202840
7,1.585563e+09,2020-03-30 10:17:48.202840
8,1.585563e+09,2020-03-30 10:18:14.202840
9,1.585564e+09,2020-03-30 10:18:47.202840


In [61]:
df['DIFF'] = df['TIMESTAMP'] - df['TIMESTAMP'].shift(1)
df

,TIMESTAMP,DATETIME,DIFF
0,1.585563e+09,2020-03-30 10:14:13.202840,NaN
1,1.585563e+09,2020-03-30 10:14:50.202840,37.0
2,1.585563e+09,2020-03-30 10:15:15.202840,25.0
3,1.585563e+09,2020-03-30 10:15:38.202840,23.0
4,1.585563e+09,2020-03-30 10:16:13.202840,35.0
5,1.585563e+09,2020-03-30 10:16:37.202840,24.0
6,1.585563e+09,2020-03-30 10:17:14.202840,37.0
7,1.585563e+09,2020-03-30 10:17:48.202840,34.0
8,1.585563e+09,2020-03-30 10:18:14.202840,26.0
9,1.585564e+09,2020-03-30 10:18:47.202840,33.0


In [62]:
bk_df = df['DIFF']

In [63]:
bk_df

0      NaN
1     37.0
2     25.0
3     23.0
4     35.0
5     24.0
6     37.0
7     34.0
8     26.0
9     33.0
10    23.0
Name: DIFF, dtype: float64

In [64]:
df.iloc[1  ] = np.nan
df.iloc[4:7] = np.nan
df.iloc[9  ] = np.nan

df['DIFF'] = bk_df
df

,TIMESTAMP,DATETIME,DIFF
0,1.585563e+09,2020-03-30 10:14:13.202840,NaN
1,NaN,NaT,37.0
2,1.585563e+09,2020-03-30 10:15:15.202840,25.0
3,1.585563e+09,2020-03-30 10:15:38.202840,23.0
4,NaN,NaT,35.0
5,NaN,NaT,24.0
6,NaN,NaT,37.0
7,1.585563e+09,2020-03-30 10:17:48.202840,34.0
8,1.585563e+09,2020-03-30 10:18:14.202840,26.0
9,NaN,NaT,33.0


In [65]:
gap_start = None
gap_end   = None
gap_list  = []

for idx, row in df.iterrows():
    timestamp = row['TIMESTAMP']
    if (pd.isna(timestamp)):
        prev_timestamp = df.iloc[idx-1]['TIMESTAMP']
        if (not pd.isna(prev_timestamp)):
            gap_start = idx-1
            pass
        next_timestamp = df.iloc[idx+1]['TIMESTAMP']
        if (not pd.isna(next_timestamp)):
            gap_end = idx+1
            gap_list.append((gap_start, gap_end))
            
gap_list

[(0, 2), (3, 7), (8, 10)]

In [68]:
df.at[0, 'TIMESTAMP_PREDICTED']= df.iloc[0]['TIMESTAMP']
df.at[3, 'TIMESTAMP_PREDICTED']= df.iloc[3]['TIMESTAMP']
df.at[8, 'TIMESTAMP_PREDICTED']= df.iloc[8]['TIMESTAMP']
df

,TIMESTAMP,DATETIME,DIFF,TIMESTAMP_PREDICTED
0,1.585563e+09,2020-03-30 10:14:13.202840,NaN,1.585563e+09
1,NaN,NaT,37.0,NaN
2,1.585563e+09,2020-03-30 10:15:15.202840,25.0,NaN
3,1.585563e+09,2020-03-30 10:15:38.202840,23.0,1.585563e+09
4,NaN,NaT,35.0,NaN
5,NaN,NaT,24.0,NaN
6,NaN,NaT,37.0,NaN
7,1.585563e+09,2020-03-30 10:17:48.202840,34.0,NaN
8,1.585563e+09,2020-03-30 10:18:14.202840,26.0,1.585563e+09
9,NaN,NaT,33.0,NaN


In [69]:
for gap in gap_list:
    gap_start  = gap[0]
    gap_end    = gap[1]
    
    iter_count = gap_end - gap_start
    for i in range(0, iter_count):
        idx = gap_start+i+1
        df.at[idx, 'TIMESTAMP_PREDICTED'] = df.iloc[idx-1]['TIMESTAMP_PREDICTED']+df.iloc[idx]['DIFF']
    
df

,TIMESTAMP,DATETIME,DIFF,TIMESTAMP_PREDICTED
0,1.585563e+09,2020-03-30 10:14:13.202840,NaN,1.585563e+09
1,NaN,NaT,37.0,1.585563e+09
2,1.585563e+09,2020-03-30 10:15:15.202840,25.0,1.585563e+09
3,1.585563e+09,2020-03-30 10:15:38.202840,23.0,1.585563e+09
4,NaN,NaT,35.0,1.585563e+09
5,NaN,NaT,24.0,1.585563e+09
6,NaN,NaT,37.0,1.585563e+09
7,1.585563e+09,2020-03-30 10:17:48.202840,34.0,1.585563e+09
8,1.585563e+09,2020-03-30 10:18:14.202840,26.0,1.585563e+09
9,NaN,NaT,33.0,1.585564e+09


In [72]:
for gap in gap_list:
    gap_start = gap[0]
    gap_end   = gap[1]
    scale_min = df.iloc[gap_start]['TIMESTAMP']
    scale_max = df.iloc[gap_end  ]['TIMESTAMP']
    predicted_timestamps = df.iloc[gap_start:gap_end+1]['TIMESTAMP_PREDICTED'].values
    
    rescaled = np.interp(
        predicted_timestamps, 
        (predicted_timestamps.min(), predicted_timestamps.max()), 
        (scale_min, scale_max)
    )
    iter_count = gap_end - gap_start
    for i in range(0, iter_count+1):
        idx = gap_start+i
        df.at[idx, 'TIMESTAMP_SCALED'  ] = rescaled[i]
        df.at[idx, 'DATETIME_SCALED'   ] = datetime.utcfromtimestamp(rescaled[i])
        df.at[idx, 'DATETIME_PREDICTED'] = datetime.utcfromtimestamp(df.iloc[idx]['TIMESTAMP_PREDICTED'])
    
df

,TIMESTAMP,DATETIME,DIFF,TIMESTAMP_PREDICTED,TIMESTAMP_SCALED,DATETIME_SCALED,DATETIME_PREDICTED
0,1.585563e+09,2020-03-30 10:14:13.202840,NaN,1.585563e+09,1.585563e+09,2020-03-30 10:14:13.202840,2020-03-30 10:14:13.202840
1,NaN,NaT,37.0,1.585563e+09,1.585563e+09,2020-03-30 10:14:50.202840,2020-03-30 10:14:50.202840
2,1.585563e+09,2020-03-30 10:15:15.202840,25.0,1.585563e+09,1.585563e+09,2020-03-30 10:15:15.202840,2020-03-30 10:15:15.202840
3,1.585563e+09,2020-03-30 10:15:38.202840,23.0,1.585563e+09,1.585563e+09,2020-03-30 10:15:38.202840,2020-03-30 10:15:38.202840
4,NaN,NaT,35.0,1.585563e+09,1.585563e+09,2020-03-30 10:16:13.202840,2020-03-30 10:16:13.202840
5,NaN,NaT,24.0,1.585563e+09,1.585563e+09,2020-03-30 10:16:37.202840,2020-03-30 10:16:37.202840
6,NaN,NaT,37.0,1.585563e+09,1.585563e+09,2020-03-30 10:17:14.202840,2020-03-30 10:17:14.202840
7,1.585563e+09,2020-03-30 10:17:48.202840,34.0,1.585563e+09,1.585563e+09,2020-03-30 10:17:48.202840,2020-03-30 10:17:48.202840
8,1.585563e+09,2020-03-30 10:18:14.202840,26.0,1.585563e+09,1.585563e+09,2020-03-30 10:18:14.202840,2020-03-30 10:18:14.202840
9,NaN,NaT,33.0,1.585564e+09,1.585564e+09,2020-03-30 10:18:47.202840,2020-03-30 10:18:47.202840


In [ ]:
test_df = pd.read_csv("./out/test_reshoot_preadjustion.csv")

In [ ]:
test_df.head(15)

In [ ]:
#cnt      = 0
gap_list = []
for idx, row in test_df.iterrows():
    timestamp = row['TIMESTAMP_BK']
    if (pd.isna(timestamp)):
        prev_timestamp = test_df.iloc[idx-1]['TIMESTAMP_BK']
        if (not pd.isna(prev_timestamp)):
            gap_start = idx-1
            pass
        next_timestamp = test_df.iloc[idx+1]['TIMESTAMP_BK']
        if (not pd.isna(next_timestamp)):
            gap_end = idx+1
            gap_list.append((gap_start, gap_end))
            
            #cnt = cnt + 1
            #if cnt>5:
            #    break
            
len(gap_list)
        

In [ ]:
gap_list[:5]

In [ ]:
#cnt = 0
for gap in gap_list:
    gap_start     = gap[0]
    gap_end       = gap[1]
    scale_min     = test_df.iloc[gap_start]['TIMESTAMP_BK']
    scale_max     = test_df.iloc[gap_end  ]['TIMESTAMP_BK']
    bk_timestamps = test_df.iloc[gap_start:gap_end+1]['TIMESTAMP'].values
    
    rescaled = np.interp(
        bk_timestamps, 
        (bk_timestamps.min(), bk_timestamps.max()), 
        (scale_min, scale_max)
    )
    iter_count = gap_end - gap_start
    for i in range(0, iter_count+1):
        idx = gap_start+i
        test_df.at[idx, 'TIMESTAMP_SCALED'] = rescaled[i]
        test_df.at[idx, 'DATETIME_SCALED' ] = datetime.utcfromtimestamp(rescaled[i])
        
    #cnt = cnt+1
    #if cnt>5:
    #    break

In [ ]:
for idx, row in test_df.iterrows():
    timestamp_scaled = row['TIMESTAMP_SCALED']
    if (pd.isna(timestamp_scaled)):
        test_df.at[idx, 'TIMESTAMP_SCALED'] = row['TIMESTAMP']

In [ ]:
test_df.to_csv("./out/test_df_timestamp_adjusted.csv")
test_df[['index', 'TIMESTAMP_SCALED']].to_csv("./out/output_ready.csv", index=False)

In [ ]:
test_df.head(30)